# **✨Policy & Value Iteration**

## **📑Table of Contents**

1. [Overview](#1-overview)

2. [Policy Iteration](#2-policy-iteration-pi)
   - 2.1 [Algorithm Overview and Definition](#21-algorithm-overview-and-definition)
   - 2.2 [Algorithm Steps](#22-algorithm-steps)
   - 2.3 [Implementation](#23-implementation)
     - 2.3.1 [Environment Setup](#231-environment-setup)
     - 2.3.2 [Policy Evaluation Phase](#232-policy-evaluation-phase)
     - 2.3.3 [Policy Improvement Phase](#233-policy-improvement-phase)
     - 2.3.4 [Complete Policy Iteration Algorithm](#234-complete-policy-iteration-algorithm)
     - 2.3.5 [Visualization and Results](#235-visualization-and-results)
   - 2.4 [Policy Iteration Properties](#24-policy-iteration-properties)
     - 2.4.1 [Convergence Guarantees](#241-convergence-guarantees)
     - 2.4.2 [Policy Improvement Theorem](#242-policy-improvement-theorem)
     - 2.4.3 [Computational Complexity](#243-computational-complexity)
   - 2.5 [Advantages and Disadvantages](#25-advantages-and-disadvantages)
   - 2.6 [Example Results](#26-example-results)

3. [Value Iteration](#3-value-iteration)
   - 3.1 [Algorithm Overview and Definition](#31-algorithm-overview-and-definition)
   - 3.2 [Mathematical Foundation](#32-mathematical-foundation)
   - 3.3 [Algorithm Steps](#33-algorithm-steps)
   - 3.4 [Implementation](#34-implementation)
     - 3.4.1 [Environment Setup](#341-environment-setup)
     - 3.4.2 [Core Value Iteration Functions](#342-core-value-iteration-functions)
     - 3.4.3 [Complete Value Iteration Algorithm](#343-complete-value-iteration-algorithm)
     - 3.4.4 [Policy Extraction](#344-policy-extraction)
   - 3.5 [Convergence Analysis and Properties](#35-convergence-analysis-and-properties)
     - 3.5.1 [Convergence Properties](#351-convergence-properties)
     - 3.5.2 [Convergence Analysis Implementation](#352-convergence-analysis-implementation)
     - 3.5.3 [Computational Complexity](#353-computational-complexity)
   - 3.6 [Value Iteration vs Policy Iteration](#36-value-iteration-vs-policy-iteration)
     - 3.6.1 [Detailed Comparison](#361-detailed-comparison)
     - 3.6.2 [Performance Comparison Implementation](#362-performance-comparison-implementation)
     - 3.6.3 [Intuitive Understanding](#363-intuitive-understanding)
     - 3.6.4 [When to Use Each](#364-when-to-use-each)
   - 3.7 [Advanced Topics](#37-advanced-topics)
     - 3.7.1 [Modified Policy Iteration (Hybrid Approach)](#371-modified-policy-iteration-hybrid-approach)
     - 3.7.2 [Visualization and Results](#372-visualization-and-results)
   - 3.8 [Key Advantages and Disadvantages](#38-key-advantages-and-disadvantages)

## **🔖1 Overview**

* **Goal**: Find the **optimal policy** (π\*) that maximizes expected return in a Markov Decision Process (MDP).
* Two key algorithms:

  * **Policy Iteration (PI)** → Iterative evaluation + improvement of a policy.
  * **Value Iteration (VI)** → A faster version that combines evaluation & improvement in one step.

# **🔖2 Policy Iteration ($PI$)**

## 2.1 **Algorithm Overview and Definition**

**Policy Iteration** is a dynamic programming algorithm that finds the optimal policy by alternating between policy evaluation and policy improvement until convergence.

> **Definition:** Policy Iteration finds the optimal policy through an iterative process of policy evaluation and improvement phases, guaranteed to converge to the optimal policy π* in finite steps.

### Mathematical Foundation

The algorithm is based on the **Policy Improvement Theorem**: If $Q^{\pi}(s, \pi'(s)) \geq V^{\pi}(s)$ for all states $s$, then policy $\pi'$ is at least as good as policy $\pi$.

**Process Flow**: Initialize policy → Evaluate policy → Improve policy → Until policy stops changing → Optimal policy

***

## 2.2 **Algorithm Steps**

### Step 1: Initialize
- Start with an arbitrary policy $\pi_0$ (can be random)

### Step 2: Policy Evaluation
- Compute the **state-value function V(s)** under current policy π:
  $$V^{\pi}(s) = \sum_a \pi(a|s) \sum_{s',r} P(s'|s,a)[r + \gamma V^{\pi}(s')]$$
- Iterate until values converge

### Step 3: Policy Improvement  
- For each state, pick the action that maximizes the Q-value:
  $$\pi'(s) = \arg\max_a Q^{\pi}(s,a)$$
- Update policy by acting greedily with respect to $V^{\pi}$

### Step 4: Convergence Check
- If $\pi' = \pi$, stop (optimal policy found)
- Otherwise, set $\pi = \pi'$ and repeat from Step 2

---

## 2.3 **Implementation**

### 2.3.1 Environment Setup

In [ ]:
import gymnasium as gym
import numpy as np

# Environment Setup
env = gym.make('FrozenLake-v1', is_slippery=True)
num_states = env.observation_space.n
num_actions = env.action_space.n

gamma = 0.9  # discount factor
theta = 1e-6  # convergence threshold

terminal_state = num_states - 1

# Access transition probabilities
P = env.unwrapped.P

### 2.3.2 Policy Evaluation Phase

#### Basic State Value Computation

In [9]:
def compute_state_value(state, policy, V):
    """Computes the value of a state under the given policy."""
    if state == terminal_state:
        return 0
    
    action = policy[state]
    value = 0.0
    for prob, next_state, reward, terminated in P[state][action]:
        value += prob * (reward + gamma * V[next_state])
    return value

#### Iterative Policy Evaluation

In [10]:
def policy_evaluation_iterative(policy, env, gamma=0.9, theta=1e-6, max_iterations=1000):
    """
    Evaluate a policy by iteratively solving Bellman equation
    """
    # Initialize value function
    V = np.zeros(env.observation_space.n)
    
    for iteration in range(max_iterations):
        new_V = np.zeros_like(V)
        
        # Update value for each non-terminal state
        for state in range(env.observation_space.n - 1):
            if state in policy:
                action = policy[state]
                expected_value = 0
                
                # Compute expected value over all possible transitions
                transitions = env.unwrapped.P[state][action]
                for prob, next_state, reward, is_terminal in transitions:
                    if is_terminal:
                        expected_value += prob * reward
                    else:
                        expected_value += prob * (reward + gamma * V[next_state])
                
                new_V[state] = expected_value
        
        # Check for convergence
        delta = np.max(np.abs(new_V - V))
        V = new_V
        
        if delta < theta:
            break
    
    return V

#### Exact Policy Evaluation (Linear Algebra)

In [11]:
def policy_evaluation_exact(policy, env, gamma=0.9):
    """
    Solve policy evaluation exactly using linear algebra
    """
    n_states = env.observation_space.n - 1  # Exclude terminal state
    
    # Build system of linear equations: V = R + γPV
    # Rearrange to: (I - γP)V = R
    
    I = np.eye(n_states)
    P_matrix = np.zeros((n_states, n_states))
    R = np.zeros(n_states)
    
    for state in range(n_states):
        if state in policy:
            action = policy[state]
            transitions = env.unwrapped.P[state][action]
            
            for prob, next_state, reward, is_terminal in transitions:
                if not is_terminal and next_state < n_states:
                    P_matrix[state, next_state] += prob
                R[state] += prob * reward
    
    # Solve linear system
    A = I - gamma * P_matrix
    V_exact = np.linalg.solve(A, R)
    
    # Add terminal state value
    V_complete = np.zeros(env.observation_space.n)
    V_complete[:n_states] = V_exact
    
    return V_complete

### 2.3.3 Policy Improvement Phase

#### Q-Value Computation

In [12]:
def compute_q_value(state, action, V, env, gamma=0.9):
    """Compute Q(s,a) = sum over next states [ P(s'|s,a) * (R + gamma*V(s')) ]"""
    if state == terminal_state:
        return 0
    
    q = 0.0
    for prob, next_state, reward, terminated in P[state][action]:
        if terminated:
            q += prob * reward
        else:
            q += prob * (reward + gamma * V[next_state])
    return q

def compute_q_values_from_v(V, env, gamma=0.9):
    """Compute Q-values from state values"""
    Q = {}
    
    for state in range(env.observation_space.n - 1):
        for action in range(env.action_space.n):
            q_value = 0
            transitions = env.unwrapped.P[state][action]
            
            for prob, next_state, reward, is_terminal in transitions:
                if is_terminal:
                    q_value += prob * reward
                else:
                    q_value += prob * (reward + gamma * V[next_state])
            
            Q[(state, action)] = q_value
    
    return Q

#### Policy Improvement Function

In [13]:
def policy_improvement(V, env, gamma=0.9):
    """
    Improve policy by acting greedily with respect to value function
    """
    improved_policy = {}
    policy_stable = True
    
    for state in range(env.observation_space.n - 1):  # Exclude terminal state
        # Compute action values for all possible actions
        action_values = []
        for action in range(env.action_space.n):
            action_value = 0
            transitions = env.unwrapped.P[state][action]
            
            for prob, next_state, reward, is_terminal in transitions:
                if is_terminal:
                    action_value += prob * reward
                else:
                    action_value += prob * (reward + gamma * V[next_state])
            
            action_values.append(action_value)
        
        # Select action with highest value (greedy policy)
        best_action = np.argmax(action_values)
        improved_policy[state] = best_action
    
    return improved_policy, policy_stable

### 2.3.4 Complete Policy Iteration Algorithm

In [7]:
def policy_iteration_complete(env, gamma=0.9, max_iterations=100):
    """
    Complete policy iteration algorithm
    """
    # Initialize with random policy
    policy = {state: np.random.choice(env.action_space.n) 
              for state in range(env.observation_space.n - 1)}
    
    iteration = 0
    policy_history = []
    
    print("Starting Policy Iteration...")
    print(f"Initial policy: {policy}")
    
    while iteration < max_iterations:
        # Policy Evaluation
        print(f"\nIteration {iteration + 1}: Policy Evaluation")
        V = policy_evaluation_iterative(policy, env, gamma)
        
        # Policy Improvement  
        print(f"Iteration {iteration + 1}: Policy Improvement")
        improved_policy, policy_stable = policy_improvement(V, env, gamma)
        
        # Store policy for analysis
        policy_history.append(policy.copy())
        
        # Check for convergence
        if policy_stable or improved_policy == policy:
            print(f"Policy iteration converged after {iteration + 1} iterations")
            break
        
        policy = improved_policy
        iteration += 1
    
    return policy, V, policy_history

def analyze_policy_convergence(policy_history):
    """
    Analyze how policy changes during iteration
    """
    print("\nPolicy Evolution Analysis:")
    print("=" * 50)
    
    for i, policy in enumerate(policy_history):
        print(f"Iteration {i}: {policy}")
        
        if i > 0:
            changes = sum(1 for state in policy.keys() 
                         if policy[state] != policy_history[i-1][state])
            print(f"  States changed: {changes}")

# Run complete policy iteration
optimal_policy, optimal_V, history = policy_iteration_complete(env, gamma=0.9)
print(f"\nOptimal Policy: {optimal_policy}")
print(f"Optimal State Values: {optimal_V}")
analyze_policy_convergence(history)

Starting Policy Iteration...
Initial policy: {0: 0, 1: 3, 2: 2, 3: 0, 4: 1, 5: 1, 6: 0, 7: 2, 8: 3, 9: 3, 10: 2, 11: 2, 12: 1, 13: 1, 14: 1}

Iteration 1: Policy Evaluation
Iteration 1: Policy Improvement
Policy iteration converged after 1 iterations

Optimal Policy: {0: 0, 1: 3, 2: 2, 3: 0, 4: 1, 5: 1, 6: 0, 7: 2, 8: 3, 9: 3, 10: 2, 11: 2, 12: 1, 13: 1, 14: 1}
Optimal State Values: [0.01177125 0.0207412  0.03662707 0.01569704 0.01569704 0.
 0.06976699 0.         0.03662707 0.06976699 0.19593003 0.
 0.         0.24999999 0.58333332 0.        ]

Policy Evolution Analysis:
Iteration 0: {0: 0, 1: 3, 2: 2, 3: 0, 4: 1, 5: 1, 6: 0, 7: 2, 8: 3, 9: 3, 10: 2, 11: 2, 12: 1, 13: 1, 14: 1}


### 2.3.5 Visualization and Results

In [8]:
# Pretty print results for FrozenLake (4x4 grid)
print("✅ Optimal Policy (per state):")
policy_array = np.array([optimal_policy.get(i, 0) for i in range(16)])
print(policy_array.reshape((4, 4)))

print("\n✅ Optimal Value Function:")
print(optimal_V.reshape((4, 4)))

# Action mapping for better visualization
action_map = {0: '←', 1: '↓', 2: '→', 3: '↑'}
print("\n✅ Policy with arrows:")
policy_arrows = np.array([action_map[optimal_policy.get(i, 0)] for i in range(16)]).reshape((4, 4))
print(policy_arrows)

✅ Optimal Policy (per state):
[[0 3 2 0]
 [1 1 0 2]
 [3 3 2 2]
 [1 1 1 0]]

✅ Optimal Value Function:
[[0.01177125 0.0207412  0.03662707 0.01569704]
 [0.01569704 0.         0.06976699 0.        ]
 [0.03662707 0.06976699 0.19593003 0.        ]
 [0.         0.24999999 0.58333332 0.        ]]

✅ Policy with arrows:
[['←' '↑' '→' '←']
 ['↓' '↓' '←' '→']
 ['↑' '↑' '→' '→']
 ['↓' '↓' '↓' '←']]


## 2.4 **Policy Iteration Properties**

### 2.4.1 Convergence Guarantees

**Finite Convergence**: Policy iteration converges in finite steps
- At most $|A|^{|S|}$ possible deterministic policies
- Each iteration either improves policy or finds optimal policy
- Strict improvement until optimality reached

**Optimality**: Converges to optimal policy $\pi^*$
- Final policy satisfies Bellman optimality equation
- No further improvement possible

### 2.4.2 Policy Improvement Theorem

For any policy $\pi$ and state $s$, if we define a new policy $\pi'$ such that:
$$\pi'(s) = \arg\max_a Q^{\pi}(s,a)$$

Then $V^{\pi'}(s) \geq V^{\pi}(s)$ for all states $s$.

**Proof Intuition**: 
- Taking the best action according to current Q-values can only improve or maintain performance
- If improvement occurs in any state, it propagates through the value function
- If no improvement occurs anywhere, we have found an optimal policy

### 2.4.3 Computational Complexity

**Per Iteration**: 
- Policy Evaluation: O(|S|³) for exact solution or O(|S|²) per sweep for iterative
- Policy Improvement: O(|S||A|)

**Total Complexity**: O(k|S|³) where k is number of iterations
- k is typically small in practice (much less than $|A|^{|S|}$)
- Often converges in just a few iterations

**Time Complexity**: $O(|S|²|A|)$ per iteration, where |S| is number of states and |A| is number of actions.

***

## 2.5 **Advantages and Disadvantages**

### Advantages:
- **Guaranteed convergence** to optimal policy
- **Often fast convergence** in practice
- **Clear separation** of evaluation and improvement phases
- **Finds shortest path** to goal (optimal behavior)

### Disadvantages:
- **Requires exact policy evaluation** (computationally expensive)
- **May be slow** for large state spaces
- **Requires complete model** of environment (transition probabilities)
- **Memory intensive** for storing complete value functions

***

## 2.6 **Example Results**

For a FrozenLake environment, typical optimal policy might look like:
```
Optimal Policy: {0: 2, 1: 2, 2: 1, 3: 1, 4: 2, 5: 1, 6: 2, 7: 2}
Optimal State Values: {0: 7, 1: 8, 2: 9, 3: 7, 4: 9, 5: 10, 6: 8, 7: 10, 8: 0}
```

The algorithm typically finds the shortest path to the goal state, demonstrating significant improvement over random initial policies.

# **🔖3. Value Iteration**

## 3.1 **Algorithm Overview and Definition**

**Value Iteration** combines policy evaluation and policy improvement in a single operation, directly computing the optimal value function through repeated application of the Bellman optimality operator.

> **Definition:** Value Iteration combines policy evaluation and improvement in one step. It computes the optimal state-value function and derives the policy from it, rather than maintaining an explicit policy throughout the process.

**Key Insight**: Instead of fully evaluating a policy (like Policy Iteration), perform only one sweep of value updates followed by implicit policy improvement, making it more computationally efficient per iteration.

### Core Idea
- **Speeds up** policy iteration by combining evaluation & improvement in a single update
- **Direct approach**: Updates value estimates directly toward optimality
- **Implicit policy**: Policy is derived from values rather than maintained explicitly

***

## 3.2 **Mathematical Foundation**

### Bellman Optimality Equation
Value iteration is based on the **Bellman Optimality Equation**:
$$V^*(s) = \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V^*(s')]$$

### Bellman Optimality Operator
The Bellman optimality operator $T^*$ is defined as:
$$(T^*V)(s) = \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V(s')]$$

#### Key Mathematical Properties:

**Contraction Mapping**: When $\gamma < 1$, $T^*$ is a contraction with modulus $\gamma$
- $\|T^*V_1 - T^*V_2\|_{\infty} \leq \gamma \|V_1 - V_2\|_{\infty}$
- Guarantees unique fixed point (optimal value function)
- Ensures geometric convergence rate

**Monotonicity**: If $V_1(s) \leq V_2(s)$ for all $s$, then $(T^*V_1)(s) \leq (T^*V_2)(s)$
- Preserves ordering between value functions
- Ensures convergence from any initialization

***

## 3.3 **Algorithm Steps**

### Step 1: Initialize
- Set $V_0(s) = 0$ for all states $s$

### Step 2: Iterative Value Update
For each state, apply the Bellman optimality operator:
$$V_{k+1}(s) = \max_a \sum_{s',r} P(s'|s,a)[r + \gamma V_k(s')]$$

### Step 3: Policy Derivation (Implicit)
Policy is implicitly defined as:
$$\pi(s) = \arg\max_a \sum_{s',r} P(s'|s,a)[r + \gamma V(s')]$$

### Step 4: Convergence Check
- Continue until value updates are below a threshold: $\max_s |V_{k+1}(s) - V_k(s)| < \theta$
- Extract final policy using policy extraction

***

## 3.4 **Implementation**

### 3.4.1 Environment Setup

In [9]:
import gymnasium as gym
import numpy as np

# Environment setup
env = gym.make('FrozenLake-v1', is_slippery=True, render_mode=None)
mdp_env = env.unwrapped  # access the underlying MDP to get .P
num_states = env.observation_space.n
num_actions = env.action_space.n
terminal_state = num_states - 1  # Goal state in FrozenLake
gamma = 0.9  # Discount factor
theta = 1e-6  # Convergence threshold

# Access transition probabilities
P = env.unwrapped.P

### 3.4.2 Core Value Iteration Functions

#### Bellman Optimality Update

In [10]:
def bellman_optimality_update(V, env, gamma=0.9):
    """
    Single step of Bellman optimality operator
    """
    new_V = np.zeros_like(V)
    policy = {}
    
    for state in range(env.observation_space.n - 1):  # Exclude terminal state
        action_values = []
        
        # Compute Q-value for each action
        for action in range(env.action_space.n):
            q_value = 0
            transitions = env.unwrapped.P[state][action]
            
            for prob, next_state, reward, is_terminal in transitions:
                if is_terminal:
                    q_value += prob * reward
                else:
                    q_value += prob * (reward + gamma * V[next_state])
            
            action_values.append(q_value)
        
        # Take maximum over actions
        max_value = max(action_values)
        max_action = np.argmax(action_values)
        
        new_V[state] = max_value
        policy[state] = max_action
    
    return new_V, policy

#### Helper Functions

In [11]:
def get_max_action_and_value(state, V, env, gamma=0.9):
    """Helper function to get optimal action and value for a state."""
    Q_values = []
    for action in range(env.action_space.n):
        q_val = 0
        for prob, next_state, reward, done in env.unwrapped.P[state][action]:
            q_val += prob * (reward + gamma * V[next_state])
        Q_values.append(q_val)
    
    max_action = int(np.argmax(Q_values))
    max_q_value = Q_values[max_action]
    return max_action, max_q_value

def compute_action_value(state, action, V, env, gamma=0.9):
    """Compute Q(s,a) using current value estimates"""
    q_value = 0
    transitions = env.unwrapped.P[state][action]
    
    for prob, next_state, reward, is_terminal in transitions:
        if is_terminal:
            q_value += prob * reward
        else:
            q_value += prob * (reward + gamma * V[next_state])
    
    return q_value

### 3.4.3 Complete Value Iteration Algorithm

#### Basic Implementation

In [12]:
def value_iteration(env, gamma=0.9, threshold=1e-3, max_iterations=1000):
    """Basic value iteration algorithm."""
    # Initialize
    V = {state: 0 for state in range(env.observation_space.n)}
    policy = {state: 0 for state in range(env.observation_space.n - 1)}
    
    for iteration in range(max_iterations):
        new_V = {state: 0 for state in range(env.observation_space.n)}
        
        for state in range(env.observation_space.n - 1):  # Exclude terminal
            if state == terminal_state:
                new_V[state] = 0
                continue
            
            # Compute Q-values for all actions
            Q_values = []
            for action in range(env.action_space.n):
                q_val = 0
                for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                    q_val += prob * (reward + gamma * V[next_state])
                Q_values.append(q_val)
            
            # Take maximum
            max_q_value = max(Q_values)
            max_action = int(np.argmax(Q_values))
            new_V[state] = max_q_value
            policy[state] = max_action
        
        # Check convergence
        if all(abs(new_V[s] - V[s]) < threshold for s in range(env.observation_space.n)):
            print(f"Value iteration converged after {iteration + 1} iterations")
            break
        
        V = new_V
    
    return policy, V

#### Enhanced Implementation with Analysis

In [14]:
def value_iteration_enhanced(env, gamma=0.9, theta=1e-6, max_iterations=1000):
    """
    Enhanced value iteration algorithm with detailed tracking
    """
    # Initialize value function
    V = np.zeros(env.observation_space.n)
    
    print("Starting Value Iteration...")
    convergence_history = []
    
    for iteration in range(max_iterations):
        # Apply Bellman optimality operator
        new_V, current_policy = bellman_optimality_update(V, env, gamma)
        
        # Check for convergence
        delta = np.max(np.abs(new_V - V))
        convergence_history.append(delta)
        V = new_V
        
        if iteration % 10 == 0:  # Print progress every 10 iterations
            print(f"Iteration {iteration}: max change = {delta:.6f}")
        
        if delta < theta:
            print(f"Value iteration converged after {iteration + 1} iterations")
            break
    
    # Extract final policy
    _, final_policy = bellman_optimality_update(V, env, gamma)
    
    return V, final_policy, convergence_history

### 3.4.4 Policy Extraction

In [15]:
def extract_policy_from_values(V, env, gamma=0.9):
    """
    Extract optimal policy from optimal value function
    """
    policy = {}
    
    for state in range(env.observation_space.n - 1):  # Exclude terminal
        action_values = []
        
        for action in range(env.action_space.n):
            q_value = 0
            transitions = env.unwrapped.P[state][action]
            
            for prob, next_state, reward, is_terminal in transitions:
                if is_terminal:
                    q_value += prob * reward
                else:
                    q_value += prob * (reward + gamma * V[next_state])
            
            action_values.append(q_value)
        
        # Select action with maximum Q-value
        policy[state] = np.argmax(action_values)
    
    return policy

def verify_policy_optimality(policy, V, env, gamma=0.9):
    """
    Verify that extracted policy is optimal
    """
    print("Policy Optimality Verification:")
    print("-" * 40)
    
    violations = 0
    
    for state in range(env.observation_space.n - 1):
        # Compute value of current policy action
        policy_action = policy[state]
        policy_q = compute_action_value(state, policy_action, V, env, gamma)
        
        # Compute maximum Q-value over all actions
        max_q = max(compute_action_value(state, action, V, env, gamma) 
                   for action in range(env.action_space.n))
        
        # Check optimality condition
        if abs(policy_q - max_q) > 1e-6:
            violations += 1
            print(f"State {state}: Policy Q={policy_q:.6f}, Max Q={max_q:.6f}")
    
    if violations == 0:
        print("✓ Policy is optimal (satisfies Bellman optimality)")
    else:
        print(f"✗ Policy has {violations} optimality violations")
    
    return violations == 0

## 3.5 **Convergence Analysis and Properties**

### 3.5.1 Convergence Properties

**Geometric Convergence**: Value iteration converges at rate $\gamma$
- Error decreases by factor $\gamma$ each iteration
- Faster convergence for smaller discount factors

**Asymptotic Convergence**: Unlike Policy Iteration's finite convergence
- Values approach $V^*$ asymptotically
- Practical convergence when changes fall below threshold

### 3.5.2 Convergence Analysis Implementation

In [16]:
def analyze_convergence_rate(env, gamma=0.9, true_V=None):
    """
    Analyze convergence rate of value iteration
    """
    V = np.zeros(env.observation_space.n)
    errors = []
    
    if true_V is None:
        # Compute true optimal values using many iterations
        true_V, _, _ = value_iteration_enhanced(env, gamma, theta=1e-12, max_iterations=10000)
    
    print("Convergence Analysis:")
    print("Iteration | Max Error | Convergence Rate")
    print("-" * 40)
    
    for iteration in range(50):
        V, _ = bellman_optimality_update(V, env, gamma)
        error = np.max(np.abs(V - true_V))
        errors.append(error)
        
        # Compute convergence rate
        if iteration > 0:
            rate = errors[iteration] / errors[iteration-1] if errors[iteration-1] > 0 else 0
        else:
            rate = 0
        
        if iteration % 5 == 0:
            print(f"{iteration:9} | {error:9.6f} | {rate:9.6f}")
    
    # Theoretical vs empirical convergence rate
    theoretical_rate = gamma
    empirical_rate = np.mean([errors[i]/errors[i-1] for i in range(5, 20) if errors[i-1] > 1e-10])
    
    print(f"\nTheoretical convergence rate: {theoretical_rate:.6f}")
    print(f"Empirical convergence rate: {empirical_rate:.6f}")
    
    return errors

### 3.5.3 Computational Complexity

**Per Iteration**: O(|S|²|A|)
- For each state: compute Q-value for each action
- Each Q-value computation: sum over next states

**Total Complexity**: O(k|S|²|A|) where k is number of iterations
- k depends on desired accuracy and discount factor
- Typically many more iterations than Policy Iteration

***

## 3.6 **Value Iteration vs Policy Iteration**

### 3.6.1 Detailed Comparison

| Aspect | **Policy Iteration** | **Value Iteration** |
|--------|---------------------|-------------------|
| **Approach** | Two clear steps: (1) **Policy Evaluation** – compute $V^π$, (2) **Policy Improvement** – update policy greedily | Blends evaluation and improvement into **one step** using Bellman optimality equation |
| **Convergence** | **Finite iterations** – guaranteed to find optimal policy in finite steps | **Asymptotic convergence** – values approach $V^*$ geometrically |
| **Per Iteration Cost** | **High** – O(|S|³) for exact policy evaluation | **Low** – O(|S|²|A|) for value updates |
| **Total Iterations** | **Fewer** – makes big jumps with full policy evaluation | **More** – small incremental improvements |
| **Memory** | Must store **explicit policy** alongside value function | Policy **implicitly derived** from values |
| **Practical Use** | Best for **small/medium state spaces** | Preferred for **large/complex environments** |

### 3.6.2 Performance Comparison Implementation

In [17]:
def compare_with_policy_iteration(env, gamma=0.9):
    """
    Compare value iteration results with policy iteration
    """
    print("=" * 60)
    print("COMPARISON: Value Iteration vs Policy Iteration")
    print("=" * 60)
    
    import time
    
    # Run value iteration
    print("\n1. Running Value Iteration...")
    start_time = time.time()
    V_vi, policy_vi, history_vi = value_iteration_enhanced(env, gamma)
    vi_time = time.time() - start_time
    
    # Run policy iteration (assuming policy_iteration_complete exists)
    print("\n2. Running Policy Iteration...")
    start_time = time.time()
    policy_pi, V_pi, history_pi = policy_iteration_complete(env, gamma)
    pi_time = time.time() - start_time
    
    # Compare results
    print("\n3. Comparing Results:")
    print("-" * 30)
    
    # Compare policies
    policy_match = all(policy_vi.get(s) == policy_pi.get(s) for s in range(env.observation_space.n - 1))
    print(f"Policies identical: {policy_match}")
    
    # Compare values
    value_diff = np.max(np.abs(V_vi - V_pi))
    print(f"Maximum value difference: {value_diff:.8f}")
    
    # Compare performance
    print(f"Value Iteration time: {vi_time:.4f}s, iterations: {len(history_vi)}")
    print(f"Policy Iteration time: {pi_time:.4f}s, iterations: {len(history_pi)}")
    
    print(f"\nValue Iteration Policy: {policy_vi}")
    print(f"Policy Iteration Policy: {policy_pi}")
    
    return V_vi, policy_vi, V_pi, policy_pi

### 3.6.3 Intuitive Understanding

**Policy Iteration = "Think hard, act big"**
- Each step is expensive but fewer steps needed
- Complete policy evaluation ensures big improvements

**Value Iteration = "Think fast, act small"**  
- Each step is cheap but more steps required
- Incremental improvements toward optimality

### 3.6.4 When to Use Each

**Value Iteration**:
- **Large action spaces** – cheaper per iteration
- **Approximate solutions acceptable** – can stop early
- **Limited computational memory** – no policy storage
- **Online/real-time applications** – faster iterations

**Policy Iteration**:
- **Small to medium problems** – exact evaluation feasible
- **Exact solutions required** – finite convergence
- **Batch processing scenarios** – can afford expensive iterations
- **Policy stability important** – explicit policy tracking

***

## 3.7. **Advanced Topics**

### 3.7.1 Modified Policy Iteration (Hybrid Approach)

In [18]:
def modified_policy_iteration(env, gamma=0.9, k=10, theta=1e-6, max_iterations=100):
    """
    Modified policy iteration: partial policy evaluation + improvement
    Bridges gap between Policy Iteration and Value Iteration
    """
    # Initialize policy randomly
    policy = {state: np.random.choice(env.action_space.n) 
              for state in range(env.observation_space.n - 1)}
    
    V = np.zeros(env.observation_space.n)
    
    for iteration in range(max_iterations):
        # Partial policy evaluation (k steps instead of full convergence)
        for _ in range(k):
            new_V = np.zeros_like(V)
            
            for state in range(env.observation_space.n - 1):
                if state in policy:
                    action = policy[state]
                    expected_value = 0
                    
                    transitions = env.unwrapped.P[state][action]
                    for prob, next_state, reward, is_terminal in transitions:
                        if is_terminal:
                            expected_value += prob * reward
                        else:
                            expected_value += prob * (reward + gamma * V[next_state])
                    
                    new_V[state] = expected_value
            
            V = new_V
        
        # Policy improvement
        improved_policy, policy_stable = policy_improvement(V, env, gamma)
        
        if policy_stable:
            print(f"Modified policy iteration converged after {iteration + 1} iterations")
            break
        
        policy = improved_policy
    
    return policy, V

### 3.7.2 Visualization and Results

In [19]:
# Run and compare all algorithms
def run_comprehensive_comparison(env, gamma=0.9):
    """
    Run all three algorithms and compare results
    """
    print("COMPREHENSIVE ALGORITHM COMPARISON")
    print("=" * 60)
    
    # Run all three methods
    print("\n1. Value Iteration")
    V_vi, policy_vi, _ = value_iteration_enhanced(env, gamma)
    
    print("\n2. Policy Iteration") 
    policy_pi, V_pi, _ = policy_iteration_complete(env, gamma)
    
    print("\n3. Modified Policy Iteration")
    policy_mpi, V_mpi = modified_policy_iteration(env, gamma, k=5)
    
    # Display results
    print(f"\nValue Iteration Policy:     {policy_vi}")
    print(f"Policy Iteration Policy:    {policy_pi}")  
    print(f"Modified Policy Iteration:  {policy_mpi}")
    
    # Pretty print for FrozenLake (4x4 grid)
    if env.observation_space.n == 16:  # FrozenLake 4x4
        print("\n✅ Value Iteration Results:")
        policy_array = np.array([policy_vi.get(i, 0) for i in range(16)])
        print("Policy (per state):")
        print(policy_array.reshape((4, 4)))
        
        print("Value Function:")
        print(V_vi.reshape((4, 4)))
        
        # Action mapping for visualization
        action_map = {0: '←', 1: '↓', 2: '→', 3: '↑'}
        print("Policy with arrows:")
        policy_arrows = np.array([action_map[policy_vi.get(i, 0)] for i in range(16)]).reshape((4, 4))
        print(policy_arrows)
    
    return V_vi, policy_vi, V_pi, policy_pi, V_mpi, policy_mpi

# Execute comprehensive comparison
results = run_comprehensive_comparison(env, gamma=0.9)

COMPREHENSIVE ALGORITHM COMPARISON

1. Value Iteration
Starting Value Iteration...
Iteration 0: max change = 0.333333
Iteration 10: max change = 0.008062
Iteration 20: max change = 0.001935
Iteration 30: max change = 0.000536
Iteration 40: max change = 0.000139
Iteration 50: max change = 0.000036
Iteration 60: max change = 0.000009
Iteration 70: max change = 0.000002
Value iteration converged after 78 iterations

2. Policy Iteration
Starting Policy Iteration...
Initial policy: {0: 1, 1: 2, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 2, 8: 0, 9: 3, 10: 2, 11: 0, 12: 1, 13: 1, 14: 2}

Iteration 1: Policy Evaluation
Iteration 1: Policy Improvement
Policy iteration converged after 1 iterations

3. Modified Policy Iteration
Modified policy iteration converged after 1 iterations

Value Iteration Policy:     {0: np.int64(0), 1: np.int64(3), 2: np.int64(0), 3: np.int64(3), 4: np.int64(0), 5: np.int64(0), 6: np.int64(0), 7: np.int64(0), 8: np.int64(3), 9: np.int64(1), 10: np.int64(0), 11: np.int64(0), 12:

## 3.8 **Key Advantages and Disadvantages**

### Advantages:
- **Computationally efficient per iteration** – $O(|S|²|A|)$ vs $O(|S|³)$
- **Memory efficient** – no explicit policy storage required
- **Flexible stopping** – can terminate early for approximate solutions
- **Scalable** – works well with large state spaces
- **Same optimal result** as Policy Iteration

### Disadvantages:
- **More total iterations** required for convergence
- **Asymptotic convergence** – never truly reaches optimality
- **Less intuitive** – policy changes implicitly
- **Threshold dependent** – convergence criteria affects solution quality